In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
# datapath = "data/"
# all_embeddings = []
# all_parametric = []
# for i in trange(2048):
#     dir = datapath + f"batch_{i}/"
#     embeddings = pd.read_csv(dir + "embeddings.csv", index_col=0)
#     parametric = pd.read_csv(dir + "parametric.csv", index_col=0)
#     parametric = parametric.loc[embeddings.index,:]
#     all_embeddings.append(embeddings)
#     all_parametric.append(parametric)
# all_embeddings = pd.concat(all_embeddings)
# all_parametric = pd.concat(all_parametric)
# all_embeddings.to_csv("all_embeddings.csv")
# all_parametric.to_csv("all_parametric.csv")

all_embeddings = pd.read_csv("all_embeddings.csv", index_col=0)
all_parametric = pd.read_csv("all_parametric.csv", index_col=0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
all_parametric_scaled = scaler.fit_transform(all_parametric)

In [ ]:
all_parametric_scaled = scaler.fit_transform(all_parametric)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_parametric_scaled, all_embeddings.values, test_size=0.05, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.05, random_state=42)
x_train = torch.tensor(x_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = x_train.to(device)
x_val = x_val.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_val = y_val.to(device)
# y_test = y_test.to(device)

In [ ]:
# Residual Block class
class ResidualBlock(nn.Module):
    def __init__(self, input_size, layer_size, num_layers):
        super(ResidualBlock, self).__init__()
        self.layers = self._make_layers(input_size, layer_size, num_layers)

    def _make_layers(self, input_size, layer_size, num_layers):
        layers = []
        layers.append(nn.Linear(input_size, layer_size))
        layers.append(nn.ReLU())
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(layer_size, layer_size))
            layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(layer_size))
        return nn.Sequential(*layers)

    def forward(self, x):
        residual = x
        out = self.layers(x)
        total = out + residual
        return total


# Residual Network class
class ResidualNetwork(nn.Module):
    def __init__(self, input_size, output_size, layer_size, layers_per_block, num_blocks):
        super(ResidualNetwork, self).__init__()
        self.initial_layer = nn.Linear(input_size, layer_size)
        self.blocks = self._make_blocks(layer_size, layers_per_block, num_blocks)
        self.final_layer = nn.Linear(layer_size, output_size)

    def _make_blocks(self, layer_size, layers_per_block, num_blocks):
        blocks = []
        for _ in range(num_blocks):
            blocks.append(ResidualBlock(layer_size, layer_size, layers_per_block))
        return nn.Sequential(*blocks)

    def forward(self, x):
        out = self.initial_layer(x)
        out = self.blocks(out)
        out = self.final_layer(out)
        return out




In [ ]:
# Example usage
input_size = len(all_parametric.columns)
output_size = len(all_embeddings.columns)
layer_size = 256
layers_per_block = 2
num_blocks = 3
batch_size = 128

model = ResidualNetwork(input_size, output_size, layer_size, layers_per_block, num_blocks).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)


patience = 10
best_val_loss = float('inf')
counter = 0
num_epochs = 100

with torch.autograd.set_detect_anomaly(True):
    for epoch in range(num_epochs):
        # Shuffle data indices
        indices = np.arange(len(x_train))
        np.random.shuffle(indices)

        model.train()
        for i in range(0, len(x_train), batch_size):
            batch_indices = indices[i:i + batch_size]
            inputs = x_train[batch_indices]
            targets = y_train[batch_indices]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(x_val)
            val_loss = criterion(val_outputs, y_val)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}.")
                break

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation Loss: {val_loss.item()}")

Epoch [1/100] - Validation Loss: 0.004414236173033714
Epoch [2/100] - Validation Loss: 0.0032573577482253313
Epoch [3/100] - Validation Loss: 0.003209325484931469
Epoch [4/100] - Validation Loss: 0.002893711207434535
Epoch [5/100] - Validation Loss: 0.002873427700251341
Epoch [6/100] - Validation Loss: 0.0027453533839434385
Epoch [7/100] - Validation Loss: 0.0028661706019192934
Epoch [8/100] - Validation Loss: 0.002793510677292943
Epoch [9/100] - Validation Loss: 0.0026912232860922813
Epoch [10/100] - Validation Loss: 0.0026788562536239624
Epoch [11/100] - Validation Loss: 0.002638767706230283
Epoch [12/100] - Validation Loss: 0.002623543608933687
Epoch [13/100] - Validation Loss: 0.0026403001975268126
Epoch [14/100] - Validation Loss: 0.0026695786509662867
Epoch [15/100] - Validation Loss: 0.002836735686287284
Epoch [16/100] - Validation Loss: 0.002560640685260296
Epoch [17/100] - Validation Loss: 0.0025422503240406513
Epoch [18/100] - Validation Loss: 0.002683529630303383
Epoch [19/1

In [ ]:
targets = y_test
preds = model(x_test).cpu()

a_norm = preds / preds.norm(dim=1)[:, None]
b_norm = y_test / y_test.norm(dim=1)[:, None]
res = torch.mm(a_norm, b_norm.transpose(0,1))

d = torch.diag(res)
d_full = d.unsqueeze(dim=1).repeat(1, len(res))

print(torch.sum(d_full<res)/len(res)/(len(res)-1))
print(torch.sum(d_full.T<res)/len(res)/(len(res)-1))

from sklearn.metrics import mean_squared_error
preds_np = preds.detach().numpy()
targets_np = targets.detach().numpy()
print(mean_squared_error(targets_np, preds_np))
from sklearn.metrics import r2_score
r2_score(targets_np, preds_np)

tensor(5.9945e-05)
tensor(1.9180e-05)
0.0023499422


0.8103476978110292

In [ ]:
# torch.save(model.state_dict(), "model_small")